In [42]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import datetime as datetime
from datetime import datetime as dt
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.chrome.options import Options
import warnings
from bs4 import BeautifulSoup, SoupStrainer


warnings.filterwarnings("ignore", category=DeprecationWarning) 
dictCalander = {"January":"01" ,
"February":"02", 
"March":"03", 
"April":"04", 
"May":"05", 
"June":"06", 
"July":"07", 
"August":"08", 
"September":"09", 
"October":"10",
"November":"11",
"December":"12"}

In [15]:
def dataLoad():
    return pd.read_excel("./Location.xlsx").applymap(str.lower)

In [16]:
def cheapflightsearch(startlocation ,endlocation , startdate ,dataframe):
    airline = "";
    price = "";
    if(not((isValidLocation(dataframe,startlocation)) & (isValidLocation(dataframe,endlocation)))):
        return ("Invalid Location" , "-1")
    elif(not (isValidDateFormat(startdate))):
        return ("Invalid Date" , "-1")
    return (airline , price)

In [17]:
def isValidLocation(dataframe , location):
    return (location.lower() in dataframe['Name'].unique()) | (location.lower() in dataframe['Code'].unique())

In [18]:
def isValidDateFormat(dateValue):
    format = "%Y-%m-%d"
    present = dt.now().strftime(format)
    present = datetime.datetime.strptime(present, format)
    if dateValue is None:
        return False
    else:
        try:
            past = datetime.datetime.strptime(dateValue, format)
            if past>present:
                return True
            else:
                return False
        except ValueError:
            return False

In [19]:
def markupDetails(element,dayCheck):
    rowValue=0
    colvalue=0
    flag = False
    soup = BeautifulSoup(element, "lxml")
    dayPicker =soup.find_all("div", {"class": "DayPicker-Week"})
    for i in dayPicker:
        rowValue =rowValue+1
        colvalue =0
        a = len(i.find_all("div",{"class" : "DayPicker-Day--outside"}))
        dateInnercell =i.find_all("div",{"class" : "dateInnerCell"})
        for j in dateInnercell:
            textValue = j.find_all('p')[0].text
            if(int(textValue) == int(dayCheck)):
                colvalue=colvalue+1
                flag =True
            else:
                colvalue=colvalue+1
            if(flag):
                break
        if(flag):
            break
    element = "/html/body/div/div/div[2]/div/div/div[2]/div[1]/div[3]/div[1]/div/div/div/div[2]/div/div[2]/div[1]/div[3]/div[{}]/div[{}]".format(rowValue,a+colvalue)
    return element

In [20]:
def extractCheperFlight(element):
    soup = BeautifulSoup(element, "lxml")
    airline = soup.find('p', attrs={'class' : 'airlineName'}).text
    flightCode= soup.find('p', attrs={'class' : 'fliCode'}).text
    price= soup.find('p', attrs={'class' : 'white-space-no-wrap'}).text
    return (airline , flightCode , price)

In [21]:
def callCheapFlightStatusMMT(source,destination,startDate):
    dataframe = dataLoad()
    result = cheapflightsearch(source,destination,startDate,dataframe)
    if (result[0]==""):
        chrome_options = Options()
        chrome_options.add_experimental_option("detach", True)
        driver = webdriver.Chrome('./chromedriver', options=chrome_options)
        driver.get("https://www.makemytrip.com/")
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html')))
            driver.maximize_window()
            # From City Keys
            time.sleep(5)
            driver.find_element_by_xpath('//*[@id="fromCity"]').click()
            time.sleep(3)
            sourceFiled= driver.find_element_by_xpath('//*[@id="root"]/div/div[2]/div/div/div[2]/div[1]/div[1]/div[1]/div/div/div/input')
            sourceFiled.clear()
            sourceFiled.send_keys(source)
            time.sleep(3)
            driver.find_element_by_xpath('//*[@id="react-autowhatever-1-section-0-item-0"]/div/div[1]').click()
            
            #To City Keys
            if(len(driver.find_elements(By.XPATH , '//*[@id="root"]/div/div[2]/div/div/div[2]/div[1]/div[2]/div[1]/div/div/div[1]/input')))<=0:
                toCityElement = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="toCity"]')))
                print(toCityElement)
                toCityElement.click()
            time.sleep(3)
            destinationPath=driver.find_element_by_xpath('//*[@id="root"]/div/div[2]/div/div/div[2]/div[1]/div[2]/div[1]/div/div/div[1]/input')
            destinationPath.clear()
            destinationPath.send_keys(destination)
            time.sleep(3)
            driver.find_element_by_xpath('//*[@id="react-autowhatever-1-section-0-item-0"]/div/div[1]').click()
            
            #Departure Date Keys
            time.sleep(5)
            driver.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div[2]/div[1]/div[3]").click()
            time.sleep(3)
            currentmonth = driver.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div[2]/div[1]/div[3]/div[1]/div/div/div/div[2]/div/div[2]/div[1]/div[1]/div").text
            pastmonth = dictCalander.get(currentmonth.split()[0])
            startMonth=startDate.split("-")[1]
            startDay=startDate.split("-")[2]
            while(pastmonth!=startMonth):
                #Click Next Button to go for Net Month
                driver.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div[2]/div[1]/div[3]/div[1]/div/div/div/div[2]/div/div[1]/span[2]").click()
                time.sleep(3)
                currentmonth = driver.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div[2]/div[1]/div[3]/div[1]/div/div/div/div[2]/div/div[2]/div[1]/div[1]/div").text
                pastmonth = dictCalander.get(currentmonth.split()[0])
            time.sleep(3)
            element = driver.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div[2]/div[1]/div[3]/div[1]/div/div/div/div[2]/div/div[2]/div[1]/div[3]").get_attribute('outerHTML')
            clickablemonth = markupDetails(element , startDay.lstrip('0'))
            driver.find_element_by_xpath(clickablemonth).click()
            time.sleep(3)
            # Click Search Button
            driver.find_element_by_xpath("/html/body/div/div/div[2]/div/div/div[2]/p/a").click()
            time.sleep(3)
            currentUrl = driver.current_url
            time.sleep(3)
            if len(driver.find_elements(By.XPATH,"/html/body/div/div/div[2]/div[2]/div[2]/div"))>0:
                driver.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div[2]/div/div/div[3]/button").click()
            
            cheappriceElement=driver.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div/div[2]/div[2]/div[3]/div/div/div/div[1]/div[1]").get_attribute('outerHTML')
            cheapFlight = extractCheperFlight(cheappriceElement)
            
            driver.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div[2]/span/span").click()
            time.sleep(3)
            
            sortpriceElement=driver.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div/div[2]/div[2]/div[3]/div/div/div/div[1]/div[1]").get_attribute('outerHTML')
            sortFlight = extractCheperFlight(sortpriceElement)
            
            driver.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div[1]/span/span").click()
            time.sleep(3)
            
            firstestpriceElement=driver.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div/div[2]/div[2]/div[3]/div/div/div/div[1]/div[1]").get_attribute('outerHTML')
            firstFlight = extractCheperFlight(firstestpriceElement)
            
            flightDetails = {"Cheapest Flight for the Day":cheapFlight ,
                             "Sortest Flight for the Day" :sortFlight ,
                             "First Flight of the Day" : firstFlight}
            print(flightDetails)
        except Exception as e:
            print(e.args[0])
        finally:
            #driver.close()
            print("Executed")
    else:
        print(result)

In [22]:
callCheapFlightStatusMMT("del","ccu","2022-08-10")

{'Cheapest Flight for the Day': ('SpiceJet', 'SG 8263', '₹ 5,533'), 'Sortest Flight for the Day': ('IndiGo', '6E 353', '₹ 5,534'), 'First Flight of the Day': ('IndiGo', '6E 788', '₹ 5,534')}
Executed


In [130]:
def callCheapFlightStatusYAT(source ,destination , journeydate):
    dataframe = dataLoad()
    result = cheapflightsearch(source,destination,journeydate,dataframe)
    if (result[0]==""):
        chrome_options = Options()
        chrome_options.add_experimental_option("detach", True)
        driver = webdriver.Chrome('./chromedriver', options=chrome_options)
        driver.get("https://www.yatra.com/")
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html')))
            driver.maximize_window()
            time.sleep(5)
            # From City Keys
            driver.find_element_by_xpath('//*[@id="BE_flight_origin_city"]').click()
            time.sleep(3)
            sourceFiled= driver.find_element_by_xpath('//*[@id="BE_flight_origin_city"]')
            sourceFiled.clear()
            sourceFiled.send_keys(source)
            time.sleep(3)
            driver.find_element_by_xpath('//*[@id="BE_flight_form_wrapper"]/div[1]/div[2]/ul/li[1]/ul/li[1]/div/div/ul/div/div/div/li[1]/div[1]/p[1]').click()
            time.sleep(3)
            
            # To City Keys
            driver.find_element_by_xpath('//*[@id="BE_flight_arrival_city"]').click()
            time.sleep(3)
            destinationField= driver.find_element_by_xpath('//*[@id="BE_flight_arrival_city"]')
            destinationField.clear()
            destinationField.send_keys(destination)
            time.sleep(3)
            driver.find_element_by_xpath('//*[@id="BE_flight_form_wrapper"]/div[1]/div[2]/ul/li[1]/ul/li[3]/div/div/ul/div/div/div/li[1]/div[1]/p[1]').click()
            time.sleep(3)
            
            #Departure Date
            driver.find_element_by_xpath('//*[@id="BE_flight_origin_date"]').click()
            time.sleep(3)
            #dateJourney =(journeydate.split("-")[2]) + ""+([k for k,v in dictCalander.items() if v == "2022-08-10".split("-")[1]][0][:3])+"' "+(journeydate.split("-")[0][2:4])
            dateJourney =(journeydate.split("-")[2])+"/"+(journeydate.split("-")[1])+"/"+(journeydate.split("-")[0])
            newDateJourney = dateJourney.replace('/', '%2F')
            print(dateJourney)
            print(newDateJourney)
            driver.find_element_by_id(dateJourney).click()
            time.sleep(3)
            
            driver.find_element_by_xpath('//*[@id="BE_flight_flsearch_btn"]').click()
            currentURL =driver.current_url
            print(currentURL)
            if((driver.find_element_by_xpath('/html/body/h1').text)=="Access Denied"):
                currentURL=currentURL.replace(dateJourney,newDateJourney)
                print(currentURL)
                driver.close()
            driver = webdriver.Chrome('./chromedriver', options=chrome_options)
            driver.get(currentURL)
            driver.maximize_window()
            WebDriverWait(driver,35).until(EC.presence_of_element_located((By.XPATH, '//*[@id="Flight-APP"]')))
            print("Data Loaded")
        except Exception as e:
            print(e.args[0])
        finally:
            #driver.close()
            print("Executed")
    else:
        print(result)

In [128]:
callCheapFlightStatusYAT("del","ccu","2022-08-10")

10/08/2022
10%2F08%2F2022
https://flight.yatra.com/air-search-ui/dom2/trigger?type=O&viewName=normal&flexi=0&noOfSegments=1&origin=DEL&originCountry=IN&destination=CCU&destinationCountry=IN&flight_depart_date=10/08/2022&ADT=1&CHD=0&INF=0&class=Economy&source=fresco-home&unqvaldesktop=258387623820
https://flight.yatra.com/air-search-ui/dom2/trigger?type=O&viewName=normal&flexi=0&noOfSegments=1&origin=DEL&originCountry=IN&destination=CCU&destinationCountry=IN&flight_depart_date=10%2F08%2F2022&ADT=1&CHD=0&INF=0&class=Economy&source=fresco-home&unqvaldesktop=258387623820
Data Loaded
Executed
